In [1]:
from nnsight import LanguageModel
from nnsight.edit import Edit
from nnsight.util import WrapperModule
from nnsight.edit import print_gm
import torch

/share/u/caden/.conda/envs/interp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = LanguageModel("openai-community/gpt2", device_map="cuda:0", dispatch=True)

# with model.trace("hello", scan=False, validate=False):
#     test = model.transformer.h[0].attn.output.save()

In [3]:
# print_gm(model.transformer.h[0].attn)

In [4]:
class EditModule(torch.nn.Module):

    def forward(self, *args, **kwargs):
        if len(args) == 1:
            args = args[0]
        
        return args
    
edit = Edit(
    model._envoy.transformer.h[0].attn._module_path, 
    "query", 
    "query_wrapper",
    WrapperModule()
)

attn_edit = Edit(
    model._envoy.transformer.h[0].attn._module_path, 
    "attn_weights", 
    "wrapper",
    WrapperModule(),
    instance = 1,
)

# class WrapperModule(torch.nn.Module):
#     """Simple torch module which passes it's input through. Useful for hooking.
#     If there is only one argument, returns the first element.
#     """

#     def forward(self, *args, **kwargs):
#         if len(args) == 1:
#             args = args[0]

#         return args
    
# wrapper_edit = Edit(
#     model._envoy.transformer.h[3].attn._module_path, 
#     "query", 
#     "query_wrapper",
#     WrapperModule()
# )


# edits = [edit, wrapper_edit]
edits = [attn_edit]

model.load_edits(edits)

In [5]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
          (wrapper): WrapperModule()
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1-11): 11 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, 

In [6]:
with model.trace("empty", scan=False, validate=False):
    model.transformer.h[0].attn.wrapper.output
    out_two = model.transformer.h[0].attn.output.save()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [7]:
out_two.shape

(torch.Size([1, 1, 768]),
 (torch.Size([1, 12, 1, 64]), torch.Size([1, 12, 1, 64])))